In [ ]:
# default_exp models

## models

In [ ]:
#hide
from torchtools.core import *

In [ ]:
#export
import torch.nn as nn
import torch as torch
import torch.nn.functional as F

In [ ]:
torch.__version__

'1.3.1'

In [ ]:
torch.sigmoid(torch.tensor([-2.])) * (1 - -1) + -1

tensor([-0.7616])

In [ ]:
#export
# This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:

# Fawaz, H. I., Lucas, B., Forestier, G., Pelletier, C., Schmidt, D. F., Weber, J., ... & Petitjean, F. (2019). InceptionTime: Finding AlexNet for Time Series Classification. arXiv preprint arXiv:1909.04939.
# Official InceptionTime tensorflow implementation: https://github.com/hfawaz/InceptionTime


def noop(x):
    return x

def shortcut(c_in, c_out):
    return nn.Sequential(*[nn.Conv1d(c_in, c_out, kernel_size=1), 
                           nn.BatchNorm1d(c_out)])
    
class Inception(nn.Module):
    def __init__(self, c_in, bottleneck=32, ks=40, nb_filters=32):

        super().__init__()
        self.bottleneck = nn.Conv1d(c_in, bottleneck, 1) if bottleneck and c_in > 1 else noop
        mts_feat = bottleneck or c_in
        conv_layers = []
        kss = [ks // (2**i) for i in range(3)]
        # ensure odd kss until nn.Conv1d with padding='same' is available in pytorch 1.3
        kss = [ksi if ksi % 2 != 0 else ksi - 1 for ksi in kss]  
        for i in range(len(kss)):
            conv_layers.append(
                nn.Conv1d(mts_feat, nb_filters, kernel_size=kss[i], padding=kss[i] // 2))
        self.conv_layers = nn.ModuleList(conv_layers)
        self.maxpool = nn.MaxPool1d(3, stride=1, padding=1)
        self.conv = nn.Conv1d(c_in, nb_filters, kernel_size=1)
        self.bn = nn.BatchNorm1d(nb_filters * 4)
        self.act = nn.ReLU()

    def forward(self, x):
        input_tensor = x.to(torch.float)
        x = self.bottleneck(input_tensor)
        for i in range(3):
            out_ = self.conv_layers[i](x)
            if i == 0: out = out_
            else: out = torch.cat((out, out_), 1)
        mp = self.conv(self.maxpool(input_tensor))
        inc_out = torch.cat((out, mp), 1)
        return self.act(self.bn(inc_out))


class InceptionBlock(nn.Module):
    def __init__(self,c_in,bottleneck=32,ks=40,nb_filters=32,residual=True,depth=6):

        super().__init__()

        self.residual = residual
        self.depth = depth

        #inception & residual layers
        inc_mods = []
        res_layers = []
        res = 0
        for d in range(depth):
            inc_mods.append(
                Inception(c_in if d == 0 else nb_filters * 4, bottleneck=bottleneck if d > 0 else 0,ks=ks,
                          nb_filters=nb_filters))
            if self.residual and d % 3 == 2:
                res_layers.append(shortcut(c_in if res == 0 else nb_filters * 4, nb_filters * 4))
                res += 1
            else: res_layer = res_layers.append(None)
        self.inc_mods = nn.ModuleList(inc_mods)
        self.res_layers = nn.ModuleList(res_layers)
        self.act = nn.ReLU()
        
    def forward(self, x):
        res = x
        for d, l in enumerate(range(self.depth)):
            x = self.inc_mods[d](x)
            if self.residual and d % 3 == 2:
                res = self.res_layers[d](res)
                x += res
                res = x
                x = self.act(x)
        return x

In [ ]:
#export    
class InceptionTime(nn.Module):
    def __init__(self,c_in,c_out,bottleneck=32,ks=40,nb_filters=32,residual=True,depth=6):
        super().__init__()
        self.block = InceptionBlock(c_in,bottleneck=bottleneck,ks=ks,nb_filters=nb_filters,
                                    residual=residual,depth=depth)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nb_filters * 4, c_out)

    def forward(self, x):
        x = self.block(x)
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x

In [ ]:
#export
class Sigmoid(nn.Module):
    '''
    sigmoid layer
    '''
    def __init__(self, low, high):
        super().__init__()
        self.high, self.low = high, low
        
    def forward(self, x):
        return torch.sigmoid(x)*(self.high-self.low)+self.low

In [ ]:
#export
class InceptionTimeSgmOld(nn.Module):
    '''
    add a sigmoid layer to InceptionTime to get the ouput in a certain range
    '''
    
    def __init__(self, n_in, n_out):
        super().__init__()
        nn.Sequential()
        self.inc = InceptionTime(n_in, n_out)
        self.low, self.high = -1., 1.
        
    def forward(self, x):
        return torch.sigmoid(self.inc(x)) * (self.high - self.low) + self.low
        

In [ ]:
#export
class InceptionTimeSgm(nn.Module):
    '''
    add a sigmoid layer to InceptionTime to get the ouput in a certain range
    '''
    
    def __init__(self, n_in, n_out, range=(-1,1)):
        super().__init__()
        self.mod = nn.Sequential(InceptionTime(n_in, n_out), Sigmoid(*range))
        
    def forward(self, x):
        x = x.float()
        return self.mod(x)
        

In [ ]:
#export
class InceptionTimeD(nn.Module):
    '''
    add a sigmoid layer to InceptionTime to get the ouput in a certain range
    '''
    
    def __init__(self, n_in, n_out):
        super().__init__()
        self.mod = nn.Sequential(InceptionTime(n_in, n_out), Sigmoid(-1., 1.))
        
    def forward(self, xc, xd):
        x = torch.cat([xc.float(), xd.float()], dim=-2)
        x = x.float()
#         print(f'InceptionTimeSgm dtype {x.dtype}')
        return self.mod(x)

### Development

In [ ]:
#export
class InceptionTimeVar(nn.Module):
    '''
    output mean and variance
    regression model, sigmoid for the mean output optional
    '''
    
    def __init__(self, n_in, n_out, meanrange=None):
        super().__init__()
        models  = [InceptionTime(n_in, n_out+1)]
        if meanrange:
            self.sigmoid = Sigmoid(*meanrange)
        self.mod = nn.Sequential(*models)
        
    def forward(self, x):
        x = x.float()
        output = self.mod(x)
        ## enforce positivity of sigma^2
        ##output_sig_pos = tf.log(1 + tf.exp(output_sig)) + 1e-06
        output[:,-1] = (output[:,-1].exp()+1).log_() + 1e-06
        if getattr(self, 'sigmoid', None): output[:,:-1] = self.sigmoid(output[:,:-1])
        return output
        

In [ ]:
# model sanity checks
xb = torch.randn((128,10,100))
yb = torch.rand(128,1)
model = InceptionTimeVar(10,1)

preds = model(xb)
assert preds.shape == (128,2)

In [ ]:
#export
def nll_regression(preds, y_true, c=5):
    '''
    negative log likelihood loss for regression, both mu and sigma are predicted
    
    Simple and Scalable Predictive UncertaintyEstimation using Deep Ensembles
    Balaji Lakshminarayanan, Alexander Pritzel, Charles Blundell, DeepMind

    '''
    
    s1 = 0.5*preds[:,1].log() 
    s2 = 0.5*(yb.squeeze()-preds[:,0]).pow(2).div(preds[:,1])
    loss = (s1+s2).mean() + c
    return loss

In [ ]:
#export
def nll_leaky_loss(preds, y_true, c=5, alpha=0.5):
    '''
    leaky_loss with variance
    
    Simple and Scalable Predictive UncertaintyEstimation using Deep Ensembles
    Balaji Lakshminarayanan, Alexander Pritzel, Charles Blundell, DeepMind

    '''
    
    s1 = 0.5*preds[:,1].log() 
    l1 = -F.leaky_relu(preds[:,0], alpha)*y_true.float().squeeze()
    s2 = 0.5*(l1.div(preds[:,1]+1)) ## +1 to prevent optimizing for variance, maybe just an artifical problem
    loss = (s1+s2).mean() + c
    return loss

In [ ]:
loss = nll_leaky_loss(preds, yb)

print(loss)

tensor(4.9918, grad_fn=<AddBackward0>)


In [ ]:
model_var = InceptionTimeVar(10,1,meanrange=(-1,1))

In [ ]:
#simple training loop

lr = 0.01
epochs = 10
loss_fn = nll_regression
m = model_var
loss_fn = nll_leaky_loss


for i in range(epochs):
    preds = m(xb)
    loss = loss_fn(preds, yb)
    print(loss)
    loss.backward()
    with torch.no_grad():
        for p in m.parameters():
            p.sub_(lr*p.grad)
    m.zero_grad()

tensor(4.7907, grad_fn=<AddBackward0>)
tensor(3.7414, grad_fn=<AddBackward0>)
tensor(3.1807, grad_fn=<AddBackward0>)
tensor(2.5960, grad_fn=<AddBackward0>)
tensor(1.9984, grad_fn=<AddBackward0>)
tensor(1.3569, grad_fn=<AddBackward0>)
tensor(0.5428, grad_fn=<AddBackward0>)
tensor(-0.8033, grad_fn=<AddBackward0>)
tensor(-3.9921, grad_fn=<AddBackward0>)
tensor(-15.4558, grad_fn=<AddBackward0>)


In [ ]:
unweighted_profit(m(xb)[:,0], yb)

tensor(-3.2576)

In [ ]:
model_var = InceptionTimeVar(10,1)
opt = Adam(model_var.parameters(), lr=0.001)

In [ ]:
m = model_var
for i in range(epochs):
    preds = m(xb)
    loss = nll_regression(preds, yb)
    print(loss)
    loss.backward()
    with torch.no_grad():
        opt.step()
        opt.zero_grad()
        

tensor(8916.1064, grad_fn=<AddBackward0>)
tensor(7501.5732, grad_fn=<AddBackward0>)
tensor(6539.9932, grad_fn=<AddBackward0>)
tensor(5825.1211, grad_fn=<AddBackward0>)
tensor(5261.4102, grad_fn=<AddBackward0>)
tensor(4800.5986, grad_fn=<AddBackward0>)
tensor(4416.3896, grad_fn=<AddBackward0>)
tensor(4092.2979, grad_fn=<AddBackward0>)
tensor(3815.1409, grad_fn=<AddBackward0>)
tensor(3575.7354, grad_fn=<AddBackward0>)


In [ ]:
model_var = InceptionTimeVar(10,1,meanrange=(-10,5))
m = model_var
loss_fn = partial(nll_leaky_loss, alpha=0.5)
epochs=20

opt = torch.optim.Adam(m.parameters(), lr=1e-4)

for i in range(epochs):
    preds = m(xb)
    loss = loss_fn(preds, yb)
    print(loss)
    print(unweighted_profit(preds[:,0], yb))
    loss.backward()
    with torch.no_grad():
        opt.step()
        opt.zero_grad()
        

tensor(1.5254, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-0.0768, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-1.6221, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-3.1501, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-4.6924, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-6.2771, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-7.9320, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-9.6852, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-11.5683, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-13.6197, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-15.8777, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-18.3844, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-21.1874, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-24.3242, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-27.8178, grad_fn=<AddBackward0>)
tensor(0.)
tensor(-31.7143, grad_fn=<AddBackward0>)
tensor(-0.1123)
tensor(-36.0276, grad_fn=<AddBackward0>)
tensor(-0.2247)
tensor(-40.7211, grad_fn=<AddBackward0>)
tensor(-0.5617)
tensor(-45.8238, grad_fn=<AddBackward0>)
tensor(-0.8425)


KeyboardInterrupt: 

In [ ]:
unweighted_profit(m(xb)[:,0],yb)

tensor(0.)

In [ ]:
m(xb)

In [ ]:
import numpy as np

In [ ]:
np.round(m(xb).detach().numpy(), 2)

array([[-0.82,  0.  ],
       [-0.84,  0.  ],
       [-0.82,  0.  ],
       [-0.82,  0.  ],
       [-0.82,  0.  ],
       [-0.82,  0.  ],
       [-0.85,  0.  ],
       [-0.81,  0.  ],
       [-0.81,  0.01],
       [-0.85,  0.  ],
       [-0.82,  0.  ],
       [-0.82,  0.  ],
       [-0.84,  0.  ],
       [-0.84,  0.  ],
       [-0.84,  0.  ],
       [-0.82,  0.  ],
       [-0.84,  0.  ],
       [-0.82,  0.  ],
       [-0.83,  0.  ],
       [-0.81,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.82,  0.  ],
       [-0.86,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.82,  0.01],
       [-0.83,  0.  ],
       [-0.84,  0.  ],
       [-0.84,  0.  ],
       [-0.81,  0.  ],
       [-0.82,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.83,  0.  ],
       [-0.84,  0.  ],
       [-0.82,  0.  ],
       [-0.83,  0.  ],
       [-0.

In [ ]:
x1 = torch.zeros(128,1)
x2 = torch.ones(128,1)

In [ ]:
x_p = torch.cat([x1,x2], dim=1)

In [ ]:
nll_regression(x_p, yb)

tensor(5.4584)

In [ ]:
t=torch.randn(100)
torch.sigmoid(t)*(1.3-0.9)+0.9


tensor([0.9803, 1.0653, 1.1313, 1.2034, 1.0856, 1.1149, 0.9548, 1.2650, 1.1225,
        1.0263, 1.1108, 1.1613, 1.0342, 1.1508, 1.1242, 1.0461, 1.1593, 1.0856,
        1.0512, 1.1492, 1.0317, 1.1062, 1.0516, 1.2052, 1.0718, 1.2382, 1.0609,
        0.9224, 1.2014, 1.1770, 1.2056, 1.0383, 0.9821, 1.0533, 1.1642, 1.0345,
        1.0596, 1.0757, 1.0150, 1.1848, 0.9880, 1.1780, 1.0803, 0.9860, 1.0961,
        1.2026, 1.0471, 1.1474, 1.0199, 0.9712, 1.0827, 1.1966, 0.9657, 1.1112,
        1.1972, 1.0153, 1.0547, 1.1979, 1.0110, 1.1318, 0.9660, 1.1317, 1.0288,
        1.0343, 1.1463, 1.1485, 1.2269, 1.1639, 1.0283, 1.0596, 1.1076, 1.0431,
        1.1613, 1.1897, 1.0978, 1.0384, 1.0461, 1.1684, 1.1513, 1.1601, 0.9469,
        1.1904, 1.1193, 1.2421, 1.1287, 1.2092, 1.0505, 0.9479, 1.1306, 1.2492,
        0.9976, 1.1380, 1.1620, 1.0202, 1.1837, 1.1023, 1.0340, 1.1576, 1.1534,
        1.1139])

In [ ]:
torch.sigmoid(t)

tensor([0.5913, 0.7275, 0.6183, 0.5189, 0.6616, 0.6775, 0.7000, 0.7109, 0.5841,
        0.6315, 0.7232, 0.5411, 0.6829, 0.7191, 0.5508, 0.5162, 0.7131, 0.7207,
        0.5158, 0.7128, 0.6950, 0.5704, 0.6903, 0.7302, 0.5538, 0.5747, 0.6909,
        0.6565, 0.6839, 0.5790, 0.7144, 0.6901, 0.7021, 0.5765, 0.6108, 0.6747,
        0.7227, 0.7173, 0.7185, 0.5146, 0.6119, 0.7285, 0.6731, 0.7155, 0.5086,
        0.6942, 0.6788, 0.5290, 0.5611, 0.7158, 0.6941, 0.6132, 0.6277, 0.6278,
        0.6790, 0.6894, 0.5219, 0.5740, 0.7114, 0.7063, 0.6639, 0.5869, 0.6681,
        0.6533, 0.6566, 0.6462, 0.6430, 0.7135, 0.6631, 0.5016, 0.6098, 0.6257,
        0.7183, 0.7010, 0.7022, 0.6265, 0.6993, 0.7100, 0.6913, 0.5921, 0.6228,
        0.7095, 0.5008, 0.6969, 0.5941, 0.5423, 0.7106, 0.6369, 0.6663, 0.6127,
        0.6994, 0.5936, 0.6483, 0.5113, 0.5580, 0.5715, 0.5325, 0.6815, 0.6862,
        0.6572])

In [ ]:
F.sigmoid()

In [ ]:
torch.ra

In [ ]:
def _create_random_results(size):
    high, low = 1.2, 0.9
    res=torch.randn(size)
    res = torch.sigmoid(res)*(high-low)+low
    res *= -100
    res[torch.rand(size)>0.5] = 100.
    return res

In [ ]:
yb = _create_random_results((128,1))

In [ ]:
nll_leaky_loss(preds, yb)

tensor(38.1166, grad_fn=<AddBackward0>)

In [ ]:
yb.mean()

tensor(-7.1893)

In [ ]:
from functools import partial

In [ ]:
model_var = InceptionTimeVar(10,1,meanrange=(-1,1))
m = model_var
loss_fn = partial(nll_leaky_loss, alpha=0.5)
epochs=20

opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [ ]:
for i in range(epochs):
    preds = m(xb)
    loss = loss_fn(preds, yb)
    print(loss)
    print(unweighted_profit(preds[:,0], yb))
    loss.backward()
    with torch.no_grad():
        opt.step()
        opt.zero_grad()
        

tensor(-5.8582, grad_fn=<AddBackward0>)
tensor(-3.5385)
tensor(-6.6025, grad_fn=<AddBackward0>)
tensor(-3.4823)
tensor(-7.3244, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-8.0178, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-8.6825, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-9.3129, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-9.9017, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-10.4434, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-10.9383, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-11.3915, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-11.8113, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-12.2062, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-12.5798, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-12.9334, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-13.2676, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-13.5844, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-13.8866, grad_fn=<AddBackward0>)
tensor(-3.4261)
tensor(-14.1788, grad_fn=<AddBackward0

In [ ]:
for i in range(epochs):
    preds = m(xb)
    loss = loss_fn(preds, yb)
    print(loss)
    print(unweighted_profit(preds[:,0], yb))
    loss.backward()
    with torch.no_grad():
        opt.step()
        opt.zero_grad()
        

tensor(-9317.9912, grad_fn=<AddBackward0>)
tensor(-3.7070)
tensor(-30340.9004, grad_fn=<AddBackward0>)
tensor(-3.7070)
tensor(-90235.2656, grad_fn=<AddBackward0>)
tensor(-3.8755)
tensor(-214640.3750, grad_fn=<AddBackward0>)
tensor(-3.9878)
tensor(-359202.5312, grad_fn=<AddBackward0>)
tensor(-4.0440)
tensor(-397443.5000, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444806.8125, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444799.9375, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444794.8438, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444791.0625, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444788.1875, grad_fn=<AddBackward0>)
tensor(-4.1001)
tensor(-444786., grad_fn=<AddBackward0>)
tensor(-4.1563)
tensor(-444784.2500, grad_fn=<AddBackward0>)
tensor(-4.1563)
tensor(-444782.7812, grad_fn=<AddBackward0>)
tensor(-4.3248)
tensor(-444781.6250, grad_fn=<AddBackward0>)
tensor(-4.3248)
tensor(-444780.6250, grad_fn=<AddBackward0>)
tensor(-4.4933)
tensor(-444779.7500, grad_fn=<Ad

In [ ]:
m(xb)

tensor([[ 0.8102,  0.3178],
        [-0.5239,  0.6837],
        [ 0.7969,  0.3428],
        [ 0.8238,  0.3040],
        [-0.6941,  0.6855],
        [-0.5750,  0.6674],
        [ 0.8599,  0.2828],
        [-0.6396,  0.6581],
        [-0.6178,  0.6803],
        [-0.6704,  0.7044],
        [ 0.7746,  0.3350],
        [-0.4117,  0.6518],
        [ 0.7976,  0.3109],
        [-0.6089,  0.6810],
        [ 0.7839,  0.3432],
        [ 0.8260,  0.3253],
        [-0.6338,  0.7247],
        [ 0.8098,  0.2983],
        [-0.7259,  0.6800],
        [ 0.7886,  0.3059],
        [-0.4271,  0.6483],
        [ 0.8405,  0.2760],
        [ 0.8067,  0.3065],
        [ 0.7801,  0.3093],
        [-0.6602,  0.6911],
        [ 0.7929,  0.3536],
        [ 0.7634,  0.3492],
        [-0.7213,  0.6978],
        [-0.6269,  0.7063],
        [ 0.7730,  0.3216],
        [ 0.8197,  0.2993],
        [ 0.8387,  0.2986],
        [ 0.8390,  0.2937],
        [ 0.7963,  0.3075],
        [-0.6434,  0.7037],
        [-0.5897,  0

In [ ]:
model_mean = InceptionTimeSgm(10,1, range=(-1,1))
m = model_mean
loss_fn = partial(leaky_loss, alpha=0.1)
# loss_fn = F.mse_loss

epochs=20

opt = torch.optim.Adam(m.parameters(), lr=1e-3)

for i in range(epochs):
    preds = m(xb)
    loss = loss_fn(preds, yb)
    print(loss)
    print(unweighted_profit(preds, yb))
    loss.backward()
    with torch.no_grad():
        opt.step()
        opt.zero_grad()
        

tensor(-0.1196, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.1918, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.2503, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.3002, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.3442, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.3833, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.4182, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.4494, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.4771, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5019, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5238, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5433, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5605, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5757, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.5891, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.6010, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.6116, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.6209, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.6293, grad_fn=<MulBackward0>)
tensor(0.)
tensor(-0.6368, grad_fn=<MulBac

In [ ]:
np.round(preds.detach().numpy(),2)

array([[ 59.69],
       [-44.33],
       [ 32.03],
       [ 48.34],
       [-71.35],
       [-90.21],
       [ 31.71],
       [-74.66],
       [-63.18],
       [-85.05],
       [ 27.3 ],
       [-40.44],
       [ 22.68],
       [-72.75],
       [ 16.64],
       [ 59.72],
       [-79.17],
       [ 51.5 ],
       [-74.85],
       [ 15.23],
       [-46.47],
       [ 54.3 ],
       [ 39.65],
       [ 40.41],
       [-78.55],
       [ 23.05],
       [ 19.68],
       [-73.84],
       [-63.5 ],
       [ 15.34],
       [ 32.78],
       [ 43.81],
       [ 47.92],
       [ 21.04],
       [-72.07],
       [-62.32],
       [ 55.13],
       [-50.59],
       [-22.92],
       [ 25.04],
       [-52.25],
       [ 58.79],
       [-80.96],
       [-63.03],
       [ 45.97],
       [-51.07],
       [-44.96],
       [ 36.12],
       [ 38.38],
       [-35.93],
       [ 48.57],
       [-81.71],
       [-75.16],
       [-14.51],
       [ 18.78],
       [ 45.45],
       [ 64.61],
       [ 21.41],
       [-47.83

In [ ]:
weighted_profit(preds, yb, threshold=70)

tensor(-8.0606, grad_fn=<MeanBackward0>)

In [ ]:
s1 = 0.5*preds[:,1].log() 

In [ ]:
s1.mean()

tensor(-0.4077, grad_fn=<MeanBackward0>)

In [ ]:
l1 = F.leaky_relu(preds[:,0], 0.5)*yb.float().squeeze()

In [ ]:
l1.mean()

tensor(50.2426, grad_fn=<MeanBackward0>)

In [ ]:
unweighted_profit(preds[:,0], yb)

tensor(-4.0440)

In [ ]:
F.leaky_relu(preds[:,0], 0.01)

tensor([ 8.1925e-01,  2.2035e-01,  7.7802e-01,  7.5209e-01, -9.6370e-03,
        -5.4069e-03,  8.3000e-01, -5.6366e-03, -9.5591e-03,  1.5291e-02,
         8.0178e-01,  1.9101e-01,  6.7007e-01, -9.8845e-03,  5.9902e-01,
         7.5299e-01, -1.4802e-03,  7.7943e-01, -9.4004e-03,  7.8837e-01,
        -4.1210e-03,  7.9112e-01,  7.8203e-01,  7.0024e-01, -1.0000e-02,
         7.9763e-01,  7.3333e-01, -4.8869e-03, -6.0198e-03,  5.9659e-01,
         6.4868e-01,  8.3074e-01,  8.6674e-01,  8.1339e-01, -8.5559e-03,
        -8.0417e-03,  8.7010e-01, -7.2772e-03,  2.4229e-01,  6.5914e-01,
        -9.9913e-03,  8.0144e-01, -6.2689e-03, -5.1532e-03,  8.4950e-01,
        -7.8629e-04, -5.0912e-03,  8.3450e-01,  6.9821e-01, -6.6265e-03,
         8.4684e-01, -1.4540e-03, -1.1866e-03, -5.2012e-04,  6.9449e-01,
         8.3494e-01,  7.6754e-01,  8.4125e-01, -7.3983e-03,  8.4954e-01,
         6.6644e-01, -7.9540e-03, -9.4522e-03,  7.9308e-01,  5.6524e-01,
        -4.2100e-03,  4.8191e-01,  8.4140e-01,  8.2

In [ ]:
l1

tensor([ 81.9255, -22.0001,  77.8021,  75.2093,  52.1226,  25.8011,  82.9995,
         28.4714,  48.0399,  -1.5908,  80.1777, -19.7501,  67.0073,  55.0419,
         59.9020,  75.2989,   7.4788,  77.9432,  54.3332,  78.8372,  20.9156,
         79.1120,  78.2034,  70.0237,  56.1900,  79.7627,  73.3333,  27.1952,
         33.1130,  59.6591,  64.8677,  83.0743,  86.6744,  81.3389,  45.9245,
         42.3194,  87.0103,  41.0345, -25.5582,  65.9136,  55.6011,  80.1443,
         29.4647,  28.9230,  84.9501,   3.7070,  26.5617,  83.4499,  69.8207,
         33.9378,  84.6835,   7.3470,   5.6708,   2.5431,  69.4486,  83.4943,
         76.7540,  84.1252,  37.5089,  84.9543,  66.6443,  41.6723,  44.7566,
         79.3078,  56.5244,  21.7490,  48.1913,  84.1402,  82.4076,  84.7987,
         70.9222,  15.2145,  51.2575,   1.5143,   7.5915,  38.2872,  69.5466,
         15.8526,  60.6424,  48.2346,  86.2634,  89.3767,  -7.8546,  84.9656,
         85.3088,   7.4551,  -8.3118,  74.5987,  50.2686,  32.92

In [ ]:
s2 = 0.5*(l1.div(preds[:,1]))

In [ ]:
np.round(s2.detach().numpy(), 0)

array([ 5.300000e+01, -8.000000e+00,  9.100000e+01,  4.700000e+01,
        5.150000e+02,  2.200000e+01,  4.500000e+01,  2.800000e+01,
        3.400000e+02, -1.000000e+00,  9.500000e+01, -7.000000e+00,
        4.400000e+01,  1.124000e+03,  5.500000e+01,  3.700000e+01,
        3.000000e+00,  4.100000e+01,  3.570000e+02,  9.000000e+01,
        1.500000e+01,  8.000000e+01,  1.010000e+02,  9.400000e+01,
        3.717994e+06,  5.000000e+01,  4.800000e+01,  3.100000e+01,
        3.200000e+01,  4.700000e+01,  4.500000e+01,  4.100000e+01,
        4.600000e+01,  7.500000e+01,  1.430000e+02,  7.400000e+01,
        6.100000e+01,  4.700000e+01, -6.000000e+00,  5.600000e+01,
        1.050800e+04,  6.000000e+01,  5.800000e+01,  2.400000e+01,
        4.300000e+01,  2.000000e+00,  2.100000e+01,  5.900000e+01,
        4.300000e+01,  4.900000e+01,  4.000000e+01,  3.000000e+00,
        3.000000e+00,  1.000000e+00,  4.100000e+01,  4.700000e+01,
        4.800000e+01,  1.710000e+02,  5.800000e+01,  7.900000e

In [ ]:
    loss = (s1+s2).mean() + c

In [ ]:
lr = 0.01
with torch.no_grad():
    for p in model_v.parameters():
        p -= p.grad * lr

In [ ]:
preds_2 = model_v(xb)

In [ ]:
loss_2 = nll_regression(preds_2, yb)

In [ ]:
loss_2

tensor(6.6749, grad_fn=<AddBackward0>)

In [ ]:
(0.5*preds[:,1].log() + 0.5*(yb.squeeze()-preds[:,0]).pow(2).div(preds[:,1].pow(2))).mean() + 5

tensor(9.1327, grad_fn=<AddBackward0>)

In [ ]:
p = list(model.parameters())[1]

In [ ]:
p.grad

#### load in some real data